# Cleaning Data in Python

👋 Welcome to your workspace! Here, you can write and run Python code and add text in [Markdown](https://www.markdownguide.org/basic-syntax/). Below, we've imported the datasets from the course _Cleaning Data in Python_ as DataFrames as well as the packages used in the course. This is your sandbox environment: analyze the course datasets further, take notes, or experiment with code!

In [190]:
%%capture
# Install fuzzywuzzy
!pip install fuzzywuzzy

In [191]:
# Importing course packages; you can add more too!
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import missingno as msno
import fuzzywuzzy
import recordlinkage 

# Importing course datasets as DataFrames
ride_sharing = pd.read_csv('datasets/ride_sharing_new.csv', index_col = 'Unnamed: 0')
airlines = pd.read_csv('datasets/airlines_final.csv',  index_col = 'Unnamed: 0')
banking = pd.read_csv('datasets/banking_dirty.csv', index_col = 'Unnamed: 0')
restaurants = pd.read_csv('datasets/restaurants_L2.csv', index_col = 'Unnamed: 0')
restaurants_new = pd.read_csv('datasets/restaurants_L2_dirty.csv', index_col = 'Unnamed: 0')

ride_sharing.head() # Display the first five rows of this DataFrame

,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender
0,12 minutes,81,Berry St at 4th St,323,Broadway at Kearny,5480,2,1959,Male
1,24 minutes,3,Powell St BART Station (Market St at 4th St),118,Eureka Valley Recreation Center,5193,2,1965,Male
2,8 minutes,67,San Francisco Caltrain Station 2 (Townsend St...,23,The Embarcadero at Steuart St,3652,3,1993,Male
3,4 minutes,16,Steuart St at Market St,28,The Embarcadero at Bryant St,1883,1,1979,Male
4,11 minutes,22,Howard St at Beale St,350,8th St at Brannan St,4626,2,1994,Male


In [192]:
# Begin writing your own code here!

### Don't know where to start?

Try completing these tasks:
- For each DataFrame, inspect the data types of each column and, where needed, clean and convert columns into the correct data type. You should also rename any columns to have more descriptive titles.
- Identify and remove all the duplicate rows in `ride_sharing`.
- Inspect the unique values of all the columns in `airlines` and clean any inconsistencies.
- For the `airlines` DataFrame, create a new column called `International` from `dest_region`, where values representing US regions map to `False` and all other regions map to `True`.
- The `banking` DataFrame contains out of date ages. Update the `Age` column using today's date and the `birth_date` column.
- Clean the `restaurants_new` DataFrame so that it better matches the categories in the `city` and `type` column of the `restaurants` DataFrame. Afterward, given typos in restaurant names, use record linkage to generate possible pairs of rows between `restaurants` and `restaurants_new` using criteria you think is best.


In [193]:
#Find Duplicates in ride_sharing
duplicates=ride_sharing.duplicated()
print(duplicates)

0        False
1        False
2        False
3        False
4        False
         ...  
25755    False
25756    False
25757    False
25758    False
25759    False
Length: 25760, dtype: bool


In [194]:
print(airlines.head())

     id        day      airline        destination    dest_region dest_size  \
0  1351    Tuesday  UNITED INTL             KANSAI           Asia       Hub   
1   373     Friday       ALASKA  SAN JOSE DEL CABO  Canada/Mexico     Small   
2  2820   Thursday        DELTA        LOS ANGELES        West US       Hub   
3  1157    Tuesday    SOUTHWEST        LOS ANGELES        West US       Hub   
4  2992  Wednesday     AMERICAN              MIAMI        East US       Hub   

  boarding_area   dept_time  wait_min     cleanliness         safety  \
0  Gates 91-102  2018-12-31     115.0           Clean        Neutral   
1   Gates 50-59  2018-12-31     135.0           Clean      Very safe   
2   Gates 40-48  2018-12-31      70.0         Average  Somewhat safe   
3   Gates 20-39  2018-12-31     190.0           Clean      Very safe   
4   Gates 50-59  2018-12-31     559.0  Somewhat clean      Very safe   

         satisfaction  
0      Very satisfied  
1      Very satisfied  
2             Neutra

In [195]:
#check inconsistency
print(airlines['cleanliness'].unique())

['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty' 'Dirty']


In [196]:
print(airlines['dest_region'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']


In [197]:
#Resolve Inconsistency
airlines['dest_region']=airlines['dest_region'].str.lower()
airlines['dest_region']=airlines['dest_region'].replace({'eur':'europe'})

In [198]:
print(airlines['dest_region'].unique())

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']


In [199]:
#For the airlines DataFrame, create a new column called International from dest_region, where values representing US regions map to False and all other regions map to True.
dict_map={'west us':False, 'east us':False, 'midwest us':False,'asia':True, 'canada/mexico':True,'middle east':True,
 'europe':True, 'central/south america':True, 'australia/new zealand':True}
airlines['International']=airlines['dest_region'].map(dict_map)
print(airlines['International'])

0        True
1        True
2       False
3       False
4       False
        ...  
2804    False
2805    False
2806    False
2807    False
2808     True
Name: International, Length: 2477, dtype: bool


In [200]:
#The banking DataFrame contains out of date ages. Update the Age column using today's date and the birth_date column.
print(banking.head())


    cust_id  birth_date  Age  acct_amount  inv_amount   fund_A   fund_B  \
0  870A9281  1962-06-09   58     63523.31       51295  30105.0   4138.0   
1  166B05B0  1962-12-16   58     38175.46       15050   4995.0    938.0   
2  BFC13E88  1990-09-12   34     59863.77       24567  10323.0   4590.0   
3  F2158F66  1985-11-03   35     84132.10       23712   3908.0    492.0   
4  7A73F334  1990-05-17   30    120512.00       93230  12158.4  51281.0   

    fund_C   fund_D account_opened last_transaction  
0   1420.0  15632.0       02-09-18         22-02-19  
1   6696.0   2421.0       28-02-19         31-10-18  
2   8469.0   1185.0       25-04-18         02-04-18  
3   6482.0  12830.0       07-11-17         08-11-18  
4  13434.0  18383.0       14-05-18         19-07-18  


In [201]:
banking['birth_date'] = pd.to_datetime(banking['birth_date'], errors='coerce')

In [202]:
now = pd.to_datetime('now')
now

Timestamp('2022-08-12 04:57:34.224855')

In [203]:
banking['Age']=(now.year - banking['birth_date'].dt.year) - ((now.month - banking['birth_date'].dt.month) < 0)

In [204]:
print(banking['Age'])

0     60
1     59
2     31
3     36
4     32
      ..
95    48
96    32
97    37
98    52
99    29
Name: Age, Length: 100, dtype: int64


In [205]:
# Clean the restaurants_new DataFrame so that it better matches the categories in the city and type column of the restaurants DataFrame. Afterward, given typos in restaurant names, use record linkage to generate possible pairs of rows between restaurants and restaurants_new using criteria you think is best.
print(restaurants_new['type'].unique())

['american' 'californian' 'japanese' 'cajun/creole' 'hot dogs' 'diners'
 'delis' 'hamburgers' 'seafood' 'italian' 'coffee shops' 'russian'
 'steakhouses' 'mexican/tex-mex' 'noodle shops' 'mexican' 'middle eastern'
 'asian' 'vietnamese' 'health food' 'american ( new )' 'pacific new wave'
 'indonesian' 'eclectic' 'chicken' 'fast food' 'southern/soul' 'coffeebar'
 'continental' 'french ( new )' 'desserts' 'chinese' 'pizza']


In [206]:
cat_clean = set(restaurants_new['type']).difference(restaurants['type'])
cat_clean_rows = restaurants_new['type'].isin(cat_clean)
restaurants_new['type']=restaurants_new[~cat_clean_rows]['type']

In [207]:
print(restaurants_new['city'].unique())

['la' 'hollywood' 'pasadena' 'los angeles' 'new york' 'studio city'
 'venice' 'santa monica' 'mar vista' 'beverly hills' 'w. hollywood'
 'encino' 'st. boyle hts .' 'westlake village' 'westwood' 'west la'
 'chinatown' 'monterey park' 'rancho park' 'redondo beach' 'long beach'
 'marina del rey' 'culver city' 'burbank' 'century city' 'malibu'
 'seal beach' 'northridge' 'st. hermosa beach']


In [208]:
cat_clean = set(restaurants_new['city']).difference(restaurants['city'])
cat_clean_rows = restaurants_new['city'].isin(cat_clean)
restaurants_new['city']=restaurants_new[~cat_clean_rows]['city']

In [210]:
import recordlinkage
indexer = recordlinkage.Index()
indexer.block('type')
pairs = indexer.index(restaurants, restaurants_new)

In [216]:
compare_cl = recordlinkage.Compare()
compare_cl.exact('type', 'type', label='Type')
compare_cl.exact('city', 'city', label='City')
compare_cl.string('name', 'name', label='Name')
compare_cl.string('addr', 'addr', label='Address')


<Compare>

In [218]:
potential_matches = compare_cl.compute(pairs,restaurants, restaurants_new)


In [219]:
potential_matches

Type  City      Name   Address
0   0      1     0  0.120000  0.280000
    1      1     0  0.120000  0.240000
    7      1     0  0.080000  0.280000
    12     1     0  0.080000  0.250000
    13     1     0  0.120000  0.320000
...      ...   ...       ...       ...
40  18     1     0  0.181818  0.208333
281 18     1     0  0.083333  0.250000
288 18     1     0  0.285714  0.105263
302 18     1     0  0.166667  0.150000
308 18     1     0  0.120000  0.222222

[3631 rows x 4 columns]

In [220]:
matches = potential_matches[potential_matches.sum(axis = 1) >= 3]
print(matches)

        Type  City      Name   Address
0   40     1     1  0.960000  1.000000
1   28     1     1  0.888889  1.000000
2   74     1     1  0.888889  1.000000
3   1      1     1  0.833333  1.000000
4   53     1     1  0.888889  1.000000
8   43     1     1  0.888889  1.000000
9   50     1     1  0.956522  1.000000
13  7      1     1  0.875000  1.000000
14  67     1     1  0.857143  1.000000
17  12     1     1  0.909091  1.000000
20  20     1     1  0.958333  1.000000
21  20     1     1  0.333333  0.666667
    27     1     1  0.947368  1.000000
176 20     1     1  0.416667  0.705882
5   65     1     1  0.941176  1.000000
7   79     1     1  0.900000  1.000000
12  26     1     1  0.973684  1.000000
18  71     1     1  0.833333  1.000000
6   73     1     1  0.933333  1.000000
10  75     1     1  0.941176  1.000000
11  21     1     1  0.888889  1.000000
16  57     1     1  0.900000  1.000000
19  47     1     1  0.857143  1.000000
15  55     1     1  0.888889  1.000000
122 55     1     1  0.533

In [221]:
duplicate_rows = matches.index.get_level_values(1)

In [222]:
restaurants_new = restaurants_new[~restaurants_new.index.isin(duplicate_rows)]

In [224]:
full_restaurant=restaurants.append(restaurants_new)
full_restaurant

,name,addr,city,phone,type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american
...,...,...,...,...,...
76,don,1136 westwood blvd.,NaN,3102091422,italian
77,feast,1949 westwood blvd.,NaN,3104750400,NaN
78,mulberry,17040 ventura blvd.,NaN,8189068881,NaN
80,jiraffe,502 santa monica blvd,santa monica,3109176671,NaN
